# Repetition from Day 1
ToDo: Which topics to repeat? Prepare here?

# Day 2, before lunch: Gridded data analysis and plotting
Focus: netCDF files

In [17]:
import netCDF4 as nc
f = nc.Dataset('d2s2/ei.ans.2010072912.800.T.nc', 'r')

In [6]:
print('Variables:', f.variables)
print('Attributes:', f.ncattrs())

Variables: OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (720,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('latitude', <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y
unlimited dimensions: 
current shape = (361,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    units: hours since 1900-01-01 00:00:00
    calendar: standard
unlimited dimensions: time
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('t', <class 'netCDF4._netCDF4.Variable'>
int16 t(time, latitude, longitude)
    long_name: Temperature
    units: K
    add_offset: 261.2217974783569
    s

In [9]:
temp = f.variables['t']
# Retrieving the variable object versus the data
print(type(temp), type(temp[:]))

<class 'netCDF4._netCDF4.Variable'> <class 'numpy.ma.core.MaskedArray'>


In [13]:
temp = temp[:]
print(temp.shape, temp.min(), temp.max())

(1, 361, 720) 215.71340664529046 306.56727771192055


Not actually super difficult, but may be there is an even simpler way?

In [19]:
import xarray as xr
f = xr.open_dataset('d2s2/ei.ans.2010072912.800.T.nc')
temp = f.variables['t']
print(type(temp))

<class 'xarray.core.variable.Variable'>


Where is the difference to the above?